In [1]:
from agent import Agent
from topics import topics


/Users/jackson.makl@dataiku.com/agent/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

a=Agent()


In [3]:

a.chat(prompt="What challenges does airbnb face?",
       web=False,
       rag=True,
       tokens=150)
a.messages

/Users/jackson.makl@dataiku.com/agent/rag.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
100%|██████████| 4/4 [00:00<00:00, 1144.19it/s]


Loading model meta-llama/Llama-3.2-3B-Instruct on mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.34s/it]


business_&_entrepreneurs
['home rules', 'recording options', 'neighborhood details', 'host ratings', 'Airbnb competition', 'location and amenities', 'recording platforms']


[{'meta': {'timestamp': '2025-04-28 02:33:46.385414',
   'topics': ['business_&_entrepreneurs'],
   'key_words': ['home rules',
    'recording options',
    'neighborhood details',
    'host ratings',
    'Airbnb competition',
    'location and amenities',
    'recording platforms'],
   'web': False,
   'rag': True},
  'items': [{'role': 'user', 'content': 'What challenges does airbnb face?'},
   {'role': 'assistant',
    'content': 'Airbnb faces several challenges:\n\n1. **Regulatory hurdles**: The company operates in over 220 countries and regions worldwide, which means navigating complex local regulations can be challenging.\n2. **Competition from traditional lodging options**: With more people turning to online booking platforms for accommodations, competition has increased significantly, making it harder for Airbnb to stand out.\n\nRegarding price suggestions, when users search for places to stay using Airbnb\'s "similar listings" feature or filters like location, dates, and ameni

In [ ]:
a.chat(prompt="What challenges does BMW face?",
       web=False,
       rag=True,
       tokens=150)
a.messages

In [ ]:
a.chat(prompt="What is trending in music?",
       web=True,
       rag=False,
       tokens=100)
a.messages

In [ ]:
a.chat(prompt="Write a poem based our chat",use_gpt=True,messages=a.messages)

In [ ]:
a.messages